#Гугл диск


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Нейросеть

In [2]:
import os, shutil
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten 
from tensorflow.keras.applications import ResNet152V2
from keras.preprocessing.image import ImageDataGenerator, load_img

Using TensorFlow backend.


##Константы

In [3]:
FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
# определим batch_size
batch_size=15

##Загружаем предварительно обученную нейронную сеть ResNet152V2
Создадим экземпляр модели ResNet152V2

In [4]:
ResNet152V2_net = ResNet152V2(weights='imagenet', 
                  include_top=False, 
                  input_shape=input_shape)

234553344/234545216 [==============================] - 2s 0us/step


In [5]:
ResNet152V2_net.trainable = False

## Классификатор

In [6]:
model = Sequential()
# добавляем в модель сеть ResNet152V2 в качестве слоя
model.add(ResNet152V2_net)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Model)          (None, 4, 4, 2048)        58331648  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               8388864   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 4617      
Total params: 66,856,713
Trainable params: 8,525,065
Non-trainable params: 58,331,648
____________________________________

In [8]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

In [9]:
#model.load_weights("/content/drive/My Drive/Sharing-Economy-neural-network/model-cnn-1.h5")
#model.load_weights("/content/drive/My Drive/Sharing-Economy-neural-network/model-cnn-S1.h5")
model.load_weights("/content/drive/My Drive/Sharing-Economy-neural-network/model-cnn-S2.h5")



#Классификация


##Распределение по категориям

In [10]:
!mkdir imgs

In [11]:
def distribution_by_category():
    test_filenames = os.listdir("/content/imgs/")
    test_df_img = pd.DataFrame({
        'filename': test_filenames
    })
    nb_samples = test_df_img.shape[0]
    test_df_img.head() 



    test_gen = ImageDataGenerator(rescale=1./255)
    test_generator = test_gen.flow_from_dataframe(
        test_df_img, 
        "/content/imgs", 
        x_col='filename',
        y_col=None,
        class_mode=None,
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        shuffle=False
    )


    predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))

    cat = {0:'Алкоголь', 1:'Готовые блюда', 2:'Консервы', 3:'Крупы', 4:'Молочка', 5:'Мясо рыба', 6:'Орехи фрукты овощи', 7:'Соки воды', 8:'Хлеб'}
    predict_index = np.argmax(predict, axis=-1)
    for index in predict_index:
      print(cat[index])
    return predict_index


#vk_parser

In [12]:
import requests
import json
import time

In [13]:
version = '5.110'
token = '06f914db06f914db06f914db67068b6414006f906f914db58235e27d29376af1fa9f903'

###img_processing

In [14]:
def vk_respons(domain,token,version,count = 1,offset = 1):
    response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token': token,
                                'v': version,
                                'domain': domain,
                                'count': count,
                                'offset': offset
                            }
                            )
    vk_answer = response.json()
    return vk_answer

def downlaod_img(img,path):
    p = requests.get(img)
    out = open(path, "wb")
    out.write(p.content)
    out.close()
# name = 'img1.jpg'
# path = '../imgs/'+name
# downlaod_img('https://sun4-15.userapi.com/rT2PGPaYQETVJClKWc65dX2_Zj87hx-xsKcE9w/26XbpiTEEqU.jpg',path)

def tests_preparing(groups):
    for i in range(len(groups)):
        domain = groups[i]
        count = 100
        vk_answer = vk_respons(domain,token,version,count)
        for j in range(vk_answer['response']['items'].__len__()):
            post_id = vk_answer['response']['items'][j]['id']
            owner_id = vk_answer['response']['items'][j]['owner_id']
            if 'attachments' in vk_answer['response']['items'][j].keys():
                for n in range(vk_answer['response']['items'][i]['attachments'].__len__()):
                    try:
                        if 'photo' in vk_answer['response']['items'][j]['attachments'][n].keys():
                            path = '../test_imgs/'+str(owner_id)+'_'+str(post_id)+'('+str(n)+').jpg'
                            url = vk_answer['response']['items'][j]['attachments'][n]['photo']['sizes'][2]['url']
                            downlaod_img(url, path)
                    except:
                        pass
#groups = ['club_helpfoodspb','sharingfood','sharingfood_irk','foodsharing_minsk','food_sharing_spb']
#tests_preparing(groups)



###parser

In [25]:
# https://nanib0ots.pythonanywhere.com


def write_json(post_id, owner_id):
    post_id = str(post_id)
    owner_id = str(owner_id)
    flag = False
    try:
        json_d = json.load(open('groups.json'))
    except:
        json_d = {}
    if owner_id in json_d.keys():
        if json_d[owner_id] != post_id:
            flag = True
            json_d[owner_id] = post_id
            with open('groups.json', 'w') as file:
                json.dump(json_d, file, indent=2, ensure_ascii=False)
    else:
        json_d[owner_id] = post_id
        flag = True
        with open('groups.json', 'w') as file:
            json.dump(json_d, file, indent=2, ensure_ascii=False)
    return flag


def vk_respons(domain, token, version, count=1, offset=1):
    response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token': token,
                                'v': version,
                                'domain': domain,
                                'count': count,
                                'offset': offset
                            }
                            )
    vk_answer = response.json()
    return vk_answer


def vk_pars_func(data):
    for i in range(len(data)):
        domain = data[i]['group_id']
        cities = list(set(data[i]['city']))

        vk_answer = vk_respons(domain, token, version)
        print(vk_answer)
        post_id = vk_answer['response']['items'][0]['id']
        owner_id = vk_answer['response']['items'][0]['owner_id']
        writer = write_json(post_id, owner_id)
        if writer:
            print('link:', 'https://vk.com/' + domain + '?w=wall' + str(owner_id) + '_' + str(post_id))
            item = vk_answer['response']['items'][0]

            text = item['text']
            print(text)


            # Определение города
            print(cities)
            if len(cities) == 1:
                city = [cities[0]]
            else:
                city = cities
                for item_city in cities:
                    if item_city.lower() in text.lower():
                       city = [item_city]
                       break



            # Определение категории
            neural_network = False
            if 'attachments' in item.keys():
                attachments = item['attachments']
                n = 0 # индекс изображени
                for att in attachments:
                    neural_network = True
                    # Проверяем есть ли во вложениях фото
                    if att['type'] == 'photo':
                        path = './imgs/' + str(owner_id) + '_' + str(post_id) + '(' + str(n) + ').jpg'
                        url = att['photo']['sizes'][2]['url']
                        n += 1
                        downlaod_img(url, path) # загрузка изображений
            
            # Нейросеть 
            category = [] 
            folder = './imgs/'
            if neural_network:          
                category_dict = {0:'Готовые блюда', 1:'Консервы', 2:'Крупы, макаронные изделия, чай и приправы', 
                      3:'Молочная продукция', 4:'Мясное, рыба', 5:'Овощи, фрукты, орехи', 6:'Реклама', 
                      7:'Соки, напитки', 8:'Хлебобулочные и кондитерские изделия'} 
                try:              
                    predict_index = distribution_by_category()
                    if not 6 in predict_index:
                        for index in predict_index:
                            category.append(category_dict[index])
                except Exception as e: 
                    print(e) 

            # отчищаем папку с загрузками
            folder = './imgs/' 
            for the_file in os.listdir(folder): 
                file_path = os.path.join(folder, the_file) 
                try: 
                    if os.path.isfile(file_path): 
                        os.unlink(file_path) 
                    elif os.path.isdir(file_path): 
                        shutil.rmtree(file_path) 
                except Exception as e: 
                    print(e) 

                  

            #  Отправка поста боту
            send = {'link': 'https://vk.com/' + domain + '?w=wall' + str(owner_id) + '_' + str(post_id),
                    'category': category, 'city': city, 'text': text}
            bot_send = json.dumps(send)
            print(bot_send)
            requests.get('https://nanib0ots.pythonanywhere.com/bot/post', send, cookies={'parser_key': '12345678'})
            


def main():
    while (True):
        try:
            response = requests.get('https://nanib0ots.pythonanywhere.com/bot/get_data', cookies={'parser_key': '12345678'})
            if response.status_code != 200:
                print(response)
                time.sleep(5)
                main()
            data = json.loads(response.content)
            print('from db: ', data)
        except requests.exceptions.ConnectionError:
            print('ConnectionError')
            time.sleep(40)
            main()
        vk_pars_func(data)
        time.sleep(40)


if __name__ == '__main__':
    main()


from db:  [{'title': 'Фудшеринг I Отдам даром еду I Челябинск', 'city': ['Челябинск'], 'group_id': 'fudshering74'}, {'title': 'Хелпфуд. Фудшеринг. Санкт-Петербург', 'city': ['СПБ'], 'group_id': 'club_helpfoodspb'}, {'title': 'Фудшеринг Отдам даром еду', 'city': ['Москва', 'СПБ'], 'group_id': 'sharingfood'}, {'title': 'Фудшеринг Отдам даром в Иркутске', 'city': ['Иркутск'], 'group_id': 'sharingfood_irk'}, {'title': 'Наша группа', 'city': ['Иркутск'], 'group_id': 'NaniB0ots_food'}]
{'response': {'count': 143, 'items': [{'id': 6839, 'from_id': -101826852, 'owner_id': -101826852, 'date': 1592411370, 'marked_as_ads': 0, 'post_type': 'post', 'text': '[id427437472|Μария]', 'attachments': [{'type': 'photo', 'photo': {'album_id': -7, 'date': 1592411370, 'id': 457240100, 'owner_id': -101826852, 'has_tags': False, 'access_key': 'edd1c1c37d273de998', 'post_id': 6839, 'sizes': [{'height': 130, 'url': 'https://sun9-33.userapi.com/dStlkLMK5lA4303bJQ_aFnh0AeqAveyNAQ7tcg/XKHuRD4Nhe4.jpg', 'type': 'm', 

KeyboardInterrupt: ignored

In [115]:
#!rm ./imgs/*.jpg
!rm ./groups.json